In [9]:
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
import json

BASE_PATH = "../data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_norm.xlsx",
    "X_train_sin_outliers_norm.xlsx",
    "X_train_con_outliers_scal.xlsx",
    "X_train_sin_outliers_scal.xlsx"
]
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        pd.read_excel(os.path.join(BASE_PATH, path))
    )

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_norm.xlsx",
    "X_test_sin_outliers_norm.xlsx",
    "X_test_con_outliers_scal.xlsx",
    "X_test_sin_outliers_scal.xlsx"
]
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_excel(os.path.join(BASE_PATH, path))
    )

y_train = pd.read_excel(os.path.join(BASE_PATH, "y_train.xlsx"))
y_test = pd.read_excel(os.path.join(BASE_PATH, "y_test.xlsx"))

# Asegurarse de que y_train y y_test sean vectores 1D
y_train = y_train.values.ravel()  # Convertir a 1D si es necesario
y_test = y_test.values.ravel()    # Convertir a 1D si es necesario

results = []

# Definir el número de características que deseas seleccionar
k = 5  # Puedes cambiar este valor según el número de características que desees seleccionar

for index, dataset in enumerate(TRAIN_DATASETS):
    print(f"Procesando dataset {index + 1}")

    # Selección de características (usando SelectKBest)
    selector = SelectKBest(f_classif, k=k)  # Selecciona las k mejores características usando f_classif
    X_train_selected = selector.fit_transform(dataset, y_train)  # Ajuste y transformación para el conjunto de entrenamiento
    X_test_selected = selector.transform(TEST_DATASETS[index])  # Transformación para el conjunto de prueba

    # Guardar las características seleccionadas en un archivo JSON
    selected_features = dataset.columns[selector.get_support()].tolist()  # Obtener los nombres de las características seleccionadas
    selected_features_filename = f"selected_features_{index + 1}_k_{k}.json"  # Nombre del archivo JSON
    with open(f"../models/feature_selection{k}", 'w') as json_file:
        json.dump(selected_features, json_file)

    # Entrenamiento del modelo con las características seleccionadas
    model = LogisticRegression(random_state=42, max_iter=10000)  # Aumentar max_iter
    model.fit(X_train_selected, y_train)
    
    # Predicciones
    y_pred_train = model.predict(X_train_selected)
    y_pred_test = model.predict(X_test_selected)

    # Almacenar los resultados de precisión
    results.append(
        {
            "train": accuracy_score(y_train, y_pred_train),
            "test": accuracy_score(y_test, y_pred_test)
        }
    )

# Imprimir los resultados
for i, result in enumerate(results):
    print(f"Dataset {i+1} - Train Accuracy: {result['train'] * 100:.4f}%, Test Accuracy: {result['test'] * 100:.4f}%")


Procesando dataset 1
Procesando dataset 2


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [ 3  4 10 14] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Procesando dataset 3
Procesando dataset 4
Procesando dataset 5


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [ 3  4 10 14] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [ 3  4 10 14] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Procesando dataset 6
Dataset 1 - Train Accuracy: 90.9168%, Test Accuracy: 90.2865%
Dataset 2 - Train Accuracy: 89.9545%, Test Accuracy: 89.4366%
Dataset 3 - Train Accuracy: 90.9107%, Test Accuracy: 90.2987%
Dataset 4 - Train Accuracy: 89.9393%, Test Accuracy: 89.4366%
Dataset 5 - Train Accuracy: 90.8743%, Test Accuracy: 90.1287%
Dataset 6 - Train Accuracy: 89.9514%, Test Accuracy: 89.4245%


In [4]:
### SPLIT 0.4

import json
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np 

# Asumiendo que TRAIN_DATASETS y TEST_DATASETS son tus conjuntos de datos preprocesados y y_train, y_test son tus etiquetas

# Ejemplo de mejor dataset (esto depende de tu código anterior)
best_dataset_index = 2  # Esto lo debes definir según el conjunto de datos que hayas seleccionado como el mejor

# Define tu modelo y el grid search (si estás haciendo GridSearchCV)
model = LogisticRegression(random_state=42, solver='sag', tol=0.1)

# Parámetros para GridSearchCV (esto es solo un ejemplo, ajusta según tus necesidades)
param_grid = {
    "C": np.logspace(-3, 3, 5),  # Reducir a 5 valores para C
    "penalty": ["l1", "l2"],  # Probar solo l1 y l2
    "solver": ['liblinear', 'saga'],  # Limitar a 'liblinear' y 'saga', que son más rápidos
    "fit_intercept": [True, False],  # Mantener el ajuste de la intercepción
    "dual": [False],  # 'dual=True' solo con 'liblinear', por lo que se mantiene en False
    "class_weight": [None, "balanced"],  # Probar con 'None' y 'balanced'
    "multi_class": ['auto', 'ovr'],  # Reducir a combinaciones más comunes
    'tol': [1e-4, 1e-3],  # Reducir a solo dos valores para tolerancia
    'max_iter': [100, 500],  # Reducir el número de iteraciones a 2 opciones
    'warm_start': [True, False],  # Continuar desde la última solución
    'l1_ratio': np.linspace(0, 1, 3)  # Reducir a 3 valores para l1_ratio
}

# GridSearchCV con menos combinaciones de parámetros
grid = GridSearchCV(
    model, param_grid=param_grid, cv=3,  # Usar 3 pliegues para validación cruzada
    n_jobs=-1,  # Utilizar todos los núcleos del procesador
    verbose=3,  # Mayor nivel de detalle en la salida de la búsqueda
    scoring='accuracy',  # Optimizar para precisión
)

# Ajustamos el grid search al conjunto de datos seleccionado
grid.fit(TRAIN_DATASETS[best_dataset_index], y_train)

# Obtener el mejor modelo después de GridSearchCV
final_model = grid.best_estimator_

# Predicciones sobre el conjunto de entrenamiento
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset_index])

# Predicciones sobre el conjunto de prueba
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset_index])

# Calcular la exactitud
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Guardar los hiperparámetros y las métricas en un archivo JSON
data_to_save = {
    "hyperparams": grid.best_params_,  # Los mejores hiperparámetros encontrados
    "train_accuracy": train_accuracy,
    "test_accuracy": test_accuracy
}

# Guardamos los resultados en un archivo JSON
with open("../data/hyperparams/hyperparams_2.json", "w") as f:
    json.dump(data_to_save, f, indent=4)

# Imprimir las métricas de rendimiento
print(f"Training Accuracy: {train_accuracy * 100:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.4f}")

Fitting 3 folds for each of 3840 candidates, totalling 11520 fits


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Training Accuracy: 90.9801
Test Accuracy: 90.9323


In [ ]:
### SPLIT 0.4

import json
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np 

# Asumiendo que TRAIN_DATASETS y TEST_DATASETS son tus conjuntos de datos preprocesados y y_train, y_test son tus etiquetas

# Ejemplo de mejor dataset (esto depende de tu código anterior)
best_dataset_index = 2  # Esto lo debes definir según el conjunto de datos que hayas seleccionado como el mejor

# Define tu modelo y el grid search (si estás haciendo GridSearchCV)
model = LogisticRegression(random_state=42, solver='sag', tol=0.1)

# Parámetros para GridSearchCV (esto es solo un ejemplo, ajusta según tus necesidades)
param_grid = {
    "C": np.logspace(-3, 3, 5),  # Reducir a 5 valores para C
    "penalty": ["l1", "l2"],  # Probar solo l1 y l2
    "solver": ['liblinear', 'saga'],  # Limitar a 'liblinear' y 'saga', que son más rápidos
    "fit_intercept": [True, False],  # Mantener el ajuste de la intercepción
    "dual": [False],  # 'dual=True' solo con 'liblinear', por lo que se mantiene en False
    "class_weight": [None, "balanced"],  # Probar con 'None' y 'balanced'
    "multi_class": ['auto', 'ovr'],  # Reducir a combinaciones más comunes
    'tol': [1e-4, 1e-3],  # Reducir a solo dos valores para tolerancia
    'max_iter': [100, 500],  # Reducir el número de iteraciones a 2 opciones
    'warm_start': [True, False],  # Continuar desde la última solución
    'l1_ratio': np.linspace(0, 1, 3)  # Reducir a 3 valores para l1_ratio
}

# GridSearchCV con menos combinaciones de parámetros
grid = GridSearchCV(
    model, param_grid=param_grid, cv=3,  # Usar 3 pliegues para validación cruzada
    n_jobs=-1,  # Utilizar todos los núcleos del procesador
    verbose=3,  # Mayor nivel de detalle en la salida de la búsqueda
    scoring='accuracy',  # Optimizar para precisión
)

# Ajustamos el grid search al conjunto de datos seleccionado
grid.fit(TRAIN_DATASETS[best_dataset_index], y_train)

# Obtener el mejor modelo después de GridSearchCV
final_model = grid.best_estimator_

# Predicciones sobre el conjunto de entrenamiento
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset_index])

# Predicciones sobre el conjunto de prueba
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset_index])

# Calcular la exactitud
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Guardar los hiperparámetros y las métricas en un archivo JSON
data_to_save = {
    "hyperparams": grid.best_params_,  # Los mejores hiperparámetros encontrados
    "train_accuracy": train_accuracy,
    "test_accuracy": test_accuracy
}

# Guardamos los resultados en un archivo JSON
with open("../data/hyperparams/hyperparams_2.json", "w") as f:
    json.dump(data_to_save, f, indent=4)

# Imprimir las métricas de rendimiento
print(f"Training Accuracy: {train_accuracy * 100:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.4f}")

In [109]:
import json
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np 

# Asumiendo que TRAIN_DATASETS y TEST_DATASETS son tus conjuntos de datos preprocesados y y_train, y_test son tus etiquetas

# Ejemplo de mejor dataset (esto depende de tu código anterior)
best_dataset_index = 2  # Esto lo debes definir según el conjunto de datos que hayas seleccionado como el mejor

# Define tu modelo y el grid search (si estás haciendo GridSearchCV)
model = LogisticRegression(random_state=42, solver='sag', tol=0.1)

# Parámetros para GridSearchCV (esto es solo un ejemplo, ajusta según tus necesidades)
param_grid = {
    "C": np.logspace(-3, 3, 5),  # Reducir a 5 valores para C
    "penalty": ["l1", "l2"],  # Probar solo l1 y l2
    "solver": ['liblinear', 'saga'],  # Limitar a 'liblinear' y 'saga', que son más rápidos
    "fit_intercept": [True, False],  # Mantener el ajuste de la intercepción
    "dual": [False],  # 'dual=True' solo con 'liblinear', por lo que se mantiene en False
    "class_weight": [None, "balanced"],  # Probar con 'None' y 'balanced'
    "multi_class": ['auto', 'ovr'],  # Reducir a combinaciones más comunes
    'tol': [1e-4, 1e-3],  # Reducir a solo dos valores para tolerancia
    'max_iter': [100, 500],  # Reducir el número de iteraciones a 2 opciones
    'warm_start': [True, False],  # Continuar desde la última solución
    'l1_ratio': np.linspace(0, 1, 3)  # Reducir a 3 valores para l1_ratio
}

# GridSearchCV con menos combinaciones de parámetros
grid = GridSearchCV(
    model, param_grid=param_grid, cv=3,  # Usar 3 pliegues para validación cruzada
    n_jobs=-1,  # Utilizar todos los núcleos del procesador
    verbose=3,  # Mayor nivel de detalle en la salida de la búsqueda
    scoring='accuracy',  # Optimizar para precisión
)

# Ajustamos el grid search al conjunto de datos seleccionado
grid.fit(TRAIN_DATASETS[best_dataset_index], y_train)

# Obtener el mejor modelo después de GridSearchCV
final_model = grid.best_estimator_

# Predicciones sobre el conjunto de entrenamiento
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset_index])

# Predicciones sobre el conjunto de prueba
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset_index])

# Calcular la exactitud
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Guardar los hiperparámetros y las métricas en un archivo JSON
data_to_save = {
    "hyperparams": grid.best_params_,  # Los mejores hiperparámetros encontrados
    "train_accuracy": train_accuracy,
    "test_accuracy": test_accuracy
}

# Guardamos los resultados en un archivo JSON
with open("../data/hyperparams/hyperparams_2.json", "w") as f:
    json.dump(data_to_save, f, indent=4)

# Imprimir las métricas de rendimiento
print(f"Training Accuracy: {train_accuracy * 100:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.4f}")

Fitting 3 folds for each of 3840 candidates, totalling 11520 fits


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Training Accuracy: 91.0106
Test Accuracy: 90.8958


Overfitting

In [123]:
import json
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np 

# Asumiendo que TRAIN_DATASETS y TEST_DATASETS son tus conjuntos de datos preprocesados y y_train, y_test son tus etiquetas

# Ejemplo de mejor dataset (esto depende de tu código anterior)
best_dataset_index = 2  # Esto lo debes definir según el conjunto de datos que hayas seleccionado como el mejor

# Define tu modelo y el grid search (si estás haciendo GridSearchCV)
model = LogisticRegression(random_state=42, solver='sag', tol=0.1)

# Parámetros para GridSearchCV (esto es solo un ejemplo, ajusta según tus necesidades)
param_grid = {
    "C": np.logspace(-3, 3, 5),  # Reducir a 3 valores para C
    "penalty": ["l1", "l2", "elasticnet"],  # Limitar a 'l1' y 'l2'
    "solver": ['liblinear', 'saga'],  # Usar solo 'liblinear'
    "fit_intercept": [True, False],  # Mantener el ajuste de la intercepción
    "dual": [True, False],  # 'dual=True' solo con 'liblinear', por lo que se mantiene en False
    "class_weight": [None, 'balanced'],  # Usar solo 'None'
    "multi_class": ['auto', 'ovr','multinomial'],  # Usar solo 'auto'
    'tol': [1e-2, 1e-3],  # Usar solo 1e-2 para tolerancia
    'max_iter': [100, 300, 500],  # Reducir a solo 50 iteraciones
    'warm_start': [True, False],  # Evitar continuar desde la última solución
    'l1_ratio': [0.5, 0.8]  # Reducir a solo 1 valor para l1_ratio
}

# GridSearchCV con 2 pliegues para validación cruzada
grid = GridSearchCV(
    model, param_grid=param_grid, cv=5,  # Usar 2 pliegues para validación cruzada
    n_jobs=-1,  # Utilizar todos los núcleos del procesador
    verbose=3,  # Mayor nivel de detalle en la salida de la búsqueda
    scoring='accuracy',  # Optimizar para precisión
)

# Ajustamos el grid search al conjunto de datos seleccionado
grid.fit(TRAIN_DATASETS[best_dataset_index], y_train)

# Obtener el mejor modelo después de GridSearchCV
final_model = grid.best_estimator_

# Predicciones sobre el conjunto de entrenamiento
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset_index])

# Predicciones sobre el conjunto de prueba
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset_index])

# Calcular la exactitud
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Guardar los hiperparámetros y las métricas en un archivo JSON
data_to_save = {
    "hyperparams": grid.best_params_,  # Los mejores hiperparámetros encontrados
    "train_accuracy": train_accuracy,
    "test_accuracy": test_accuracy
}

# Guardamos los resultados en un archivo JSON
with open("../data/hyperparams/hyperparams_2.json", "w") as f:
    json.dump(data_to_save, f, indent=4)

# Imprimir las métricas de rendimiento
print(f"Training Accuracy: {train_accuracy * 100:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.4f}")

Fitting 5 folds for each of 17280 candidates, totalling 86400 fits


c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
50400 fits failed out of a total of 86400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4800 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Albert\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\A

Training Accuracy: 91.0167
Test Accuracy: 90.8958


Sigue estando overfitted. Por lo que nos quedaremos con el modelo sin hiperparametros. 

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
# Asumiendo que ya tienes y_test y y_pred (y_prob si calculas ROC AUC)
acc = accuracy_score(y_test, y_pred_test)
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
cm = confusion_matrix(y_test, y_pred_test)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.8942447790189413
Precision: 0.6037344398340249
Recall: 0.2996910401647786
F1 Score: 0.4005505849965588
Confusion Matrix:
 [[7074  191]
 [ 680  291]]


Accuracy: 0.8981239754720418
Precision: 0.6092307692307692
Recall: 0.3141195134849286
F1 Score: 0.4145150034891835
Confusion Matrix:
 [[14199   381]
 [ 1297   594]]

In [ ]:
conditions = [
    df['month'].isin(['jan', 'feb', 'mar']),
    df['month'].isin(['apr', 'may', 'jun']),
    df['month'].isin(['jul', 'aug', 'sep']),
    df['month'].isin(['oct', 'nov', 'dec'])
]
choices = ['Q1', 'Q2', 'Q3', 'Q4']
df['qtr'] = np.select(conditions, choices, default=0)
df['qtr'].value_counts()